## 0. Setting up

In [1]:
import torch
import torchvision
print(torch.__version__, torchvision.__version__)

2.7.0+cu118 0.22.0+cpu


In [2]:
# 1) Move “up” until you hit the folder that contains pytorch/
import os
os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))
print("Working directory is now:", os.getcwd())

Working directory is now: c:\Users\Philippa\Documents\GitHub


In [6]:
import matplotlib.pyplot as plt
from torch import nn
from torchvision import transforms
from torchinfo import summary
import sys
from pytorch.TransferLearning.python_scripts import data_setup, engine

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## 1. Getting data

In [8]:
import os
import zipfile

from pathlib import Path

import requests

# setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"
if image_path.is_dir():
    print(f"{image_path} already exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
    
    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading data...")
        f.write(request.content)
        
    # unzip the data
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping data...")
        zip_ref.extractall(image_path)
    
    # remove .zip file
    os.remove(data_path / "pizza_steak_sushi.zip")

Did not find data\pizza_steak_sushi directory, creating one...
Unzipping data...


In [9]:
# set up training and testing directories
train_dir = image_path / "train"
test_dir = image_path / "test"

## 2. Create Datasets and DataLoaders

In [10]:
# create transforms pipeline manually
manual_transforms = transforms.Compose([
    transforms.Resize((224,224)), # reshape all images to 224x224, though some models may require different sizes
    transforms.ToTensor(), # turn image values to between 0 and 1
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # a mean of [0.485, 0.456, 0.406] (across each colour channel)
                         std=[0.229, 0.224, 0.225]) # A standard deviation of [0.229, 0.224, 0.225] (across each colour channel)
])

In [11]:
# create training and testing dataloaders and get a list of class names, using data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=manual_transforms,
                                                                               batch_size=32)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x1df719d8980>,
 ['pizza', 'steak', 'sushi'])